In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_temp = pd.read_csv("../meteo_viseu_pinhao_aveiro/dados_temperatura.csv")
df_prec = pd.read_csv("../meteo_viseu_pinhao_aveiro/dados_precipitacao.csv")
df_energy = pd.read_csv("output.csv")

## IPMA Temperature

In [3]:
df_temp['Time'] = pd.to_datetime(df_temp['Time'])
df_temp

,Time,Temp_Viseu,Temp_Pinhao,Temp_Aveiro
0,2020-10-01 00:00:00,11.1,13.2,16.8
1,2020-10-01 00:10:00,11.1,13.2,16.7
2,2020-10-01 00:20:00,10.9,13.2,16.6
3,2020-10-01 00:30:00,11.3,13.3,16.5
4,2020-10-01 00:40:00,11.5,13.4,16.4
...,...,...,...,...
131323,2023-03-31 23:10:00,-990.0,-990.0,-990.0
131324,2023-03-31 23:20:00,-990.0,-990.0,-990.0
131325,2023-03-31 23:30:00,-990.0,-990.0,-990.0
131326,2023-03-31 23:40:00,-990.0,-990.0,-990.0


## IPMA Precipitation

In [4]:
df_prec

,Time,Precip_Viseu,Precip_Pinhao,Precip_Aveiro
0,2020-10-01 00:00:00,0.0,0.0,0.0
1,2020-10-01 00:10:00,0.0,0.0,0.0
2,2020-10-01 00:20:00,0.0,0.0,0.0
3,2020-10-01 00:30:00,0.0,0.0,0.0
4,2020-10-01 00:40:00,0.0,0.0,0.0
...,...,...,...,...
131323,2023-03-31 23:10:00,-990.0,-990.0,-990.0
131324,2023-03-31 23:20:00,-990.0,-990.0,-990.0
131325,2023-03-31 23:30:00,-990.0,-990.0,-990.0
131326,2023-03-31 23:40:00,-990.0,-990.0,-990.0


## Created Energy Dataset

In [5]:
df_energy

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35027,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35028,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35029,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35030,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


## Preprocessing

In [6]:
print("NaN values:\n",df_temp.isna().sum())
print("Negative values:",(df_temp.values[:,1:] < 0).sum())

NaN values:
 Time           0
Temp_Viseu     0
Temp_Pinhao    0
Temp_Aveiro    0
dtype: int64
Negative values: 41091


In [7]:
print("NaN values:\n",df_prec.isna().sum())
print("Negative values:",(df_prec.values[:,1:] < 0).sum())

NaN values:
 Time             0
Precip_Viseu     0
Precip_Pinhao    0
Precip_Aveiro    0
dtype: int64
Negative values: 16243


In [8]:
print("NaN values:\n",df_energy.isna().sum())
print("Negative values:",(df_energy.values[:,1:] < 0).sum())

NaN values:
 Time      0
PT41CV    0
PT44AL    0
PT15VP    0
PT69QG    0
PT43QL    0
PT17RW    0
PT87ZW    0
dtype: int64
Negative values: 0


### Equalize intervals

In [9]:
df_ind = df_temp.set_index('Time').iloc[:,0]
df_ind

Time
2020-10-01 00:00:00     11.1
2020-10-01 00:10:00     11.1
2020-10-01 00:20:00     10.9
2020-10-01 00:30:00     11.3
2020-10-01 00:40:00     11.5
                       ...  
2023-03-31 23:10:00   -990.0
2023-03-31 23:20:00   -990.0
2023-03-31 23:30:00   -990.0
2023-03-31 23:40:00   -990.0
2023-03-31 23:50:00   -990.0
Name: Temp_Viseu, Length: 131328, dtype: float64

In [20]:
df_split = np.array_split(df_ind, len(df_ind)/6)
df_split

[Time
 2020-10-01 00:00:00    11.1
 2020-10-01 00:10:00    11.1
 2020-10-01 00:20:00    10.9
 2020-10-01 00:30:00    11.3
 2020-10-01 00:40:00    11.5
 2020-10-01 00:50:00    11.4
 Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 01:00:00    10.8
 2020-10-01 01:10:00    10.8
 2020-10-01 01:20:00    10.6
 2020-10-01 01:30:00    10.7
 2020-10-01 01:40:00    10.9
 2020-10-01 01:50:00    10.9
 Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 02:00:00    10.9
 2020-10-01 02:10:00    11.0
 2020-10-01 02:20:00    10.9
 2020-10-01 02:30:00    10.9
 2020-10-01 02:40:00    10.9
 2020-10-01 02:50:00    10.8
 Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 03:00:00    10.7
 2020-10-01 03:10:00    10.5
 2020-10-01 03:20:00    10.5
 2020-10-01 03:30:00    10.4
 2020-10-01 03:40:00    10.4
 2020-10-01 03:50:00    10.5
 Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 04:00:00    10.4
 2020-10-01 04:10:00    10.4
 2020-10-01 04:20:00    10.5
 2020-10-01 04:30:00    10.5
 2020-10-01 04:40:

In [68]:
df_time = df_ind.resample('15T').apply(lambda x: x[0])
df_time

Time
2020-10-01 00:00:00     11.1
2020-10-01 00:15:00     10.9
2020-10-01 00:30:00     11.3
2020-10-01 00:45:00     11.4
2020-10-01 01:00:00     10.8
                       ...  
2023-03-31 22:45:00   -990.0
2023-03-31 23:00:00   -990.0
2023-03-31 23:15:00   -990.0
2023-03-31 23:30:00   -990.0
2023-03-31 23:45:00   -990.0
Freq: 15T, Name: Temp_Viseu, Length: 87552, dtype: float64

In [70]:
df_split15 = np.array_split(df_time, len(df_time)/4)
df_split15

[Time
 2020-10-01 00:00:00    11.1
 2020-10-01 00:15:00    10.9
 2020-10-01 00:30:00    11.3
 2020-10-01 00:45:00    11.4
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 01:00:00    10.8
 2020-10-01 01:15:00    10.6
 2020-10-01 01:30:00    10.7
 2020-10-01 01:45:00    10.9
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 02:00:00    10.9
 2020-10-01 02:15:00    10.9
 2020-10-01 02:30:00    10.9
 2020-10-01 02:45:00    10.8
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 03:00:00    10.7
 2020-10-01 03:15:00    10.5
 2020-10-01 03:30:00    10.4
 2020-10-01 03:45:00    10.5
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 04:00:00    10.4
 2020-10-01 04:15:00    10.5
 2020-10-01 04:30:00    10.5
 2020-10-01 04:45:00    10.5
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 05:00:00    10.6
 2020-10-01 05:15:00    10.4
 2020-10-01 05:30:00    10.4
 2020-10-01 05:45:00    10.4
 Freq: 15T, Name: Temp_Viseu, dtype: f

In [75]:
for hour in range(len(df_split)):
    df_split15[hour][0] = df_split[hour][0]
    df_split15[hour][1] = (df_split[hour][1] + df_split[hour][2]) / 2
    df_split15[hour][2] = df_split[hour][3]
    df_split15[hour][3] = (df_split[hour][4] + df_split[hour][5]) / 2
df_resampled = df_split15.copy()
df_resampled

[Time
 2020-10-01 00:00:00    11.10
 2020-10-01 00:15:00    11.00
 2020-10-01 00:30:00    11.30
 2020-10-01 00:45:00    11.45
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 01:00:00    10.8
 2020-10-01 01:15:00    10.7
 2020-10-01 01:30:00    10.7
 2020-10-01 01:45:00    10.9
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 02:00:00    10.90
 2020-10-01 02:15:00    10.95
 2020-10-01 02:30:00    10.90
 2020-10-01 02:45:00    10.85
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 03:00:00    10.70
 2020-10-01 03:15:00    10.50
 2020-10-01 03:30:00    10.40
 2020-10-01 03:45:00    10.45
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 04:00:00    10.40
 2020-10-01 04:15:00    10.45
 2020-10-01 04:30:00    10.50
 2020-10-01 04:45:00    10.50
 Freq: 15T, Name: Temp_Viseu, dtype: float64,
 Time
 2020-10-01 05:00:00    10.60
 2020-10-01 05:15:00    10.45
 2020-10-01 05:30:00    10.40
 2020-10-01 05:45:00    10.40
 Freq: 15T, Name: 

In [74]:
df_temp.head(20)

,Time,Temp_Viseu,Temp_Pinhao,Temp_Aveiro
0,2020-10-01 00:00:00,11.1,13.2,16.8
1,2020-10-01 00:10:00,11.1,13.2,16.7
2,2020-10-01 00:20:00,10.9,13.2,16.6
3,2020-10-01 00:30:00,11.3,13.3,16.5
4,2020-10-01 00:40:00,11.5,13.4,16.4
5,2020-10-01 00:50:00,11.4,13.3,16.4
6,2020-10-01 01:00:00,10.8,13.4,16.4
7,2020-10-01 01:10:00,10.8,13.5,16.4
8,2020-10-01 01:20:00,10.6,13.5,16.3
9,2020-10-01 01:30:00,10.7,13.6,16.4


In [79]:
pd.DataFrame(np.concatenate(df_resampled))

,0
0,11.10
1,11.00
2,11.30
3,11.45
4,10.80
...,...
87547,-990.00
87548,-990.00
87549,-990.00
87550,-990.00
